<a href="https://colab.research.google.com/github/meyresearch/ANI-Peptides/blob/main/demos/AMBER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime Environment Setup

Before you start, ensure you have been allocated a GPU runtime. Go to `Runtime -> Change runtime type -> GPU`

Allow kernel to restart after running this cell.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Ensure good conda/mamba installation

In [ ]:
import condacolab
condacolab.check()

Install Dependencies



*   *OpenMM* - Molecular dynamics simulation
*   *NGLView, PyTraj* - Viewing libraries



In [ ]:
!mamba install -q openmm nglview
!pip install -q pytraj

Test OpenMM

In [ ]:
!python -m openmm.testInstallation

Fetch sample peptide

In [5]:
!wget -q https://github.com/meyresearch/ANI-Peptides/raw/main/demos/aaa_equilibrated.pdb

# AMBER Simulation

In [6]:
# Import libraries

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

In [7]:
# Load sample peptide
pdb = PDBFile('aaa_equilibrated.pdb')

# Create AMBER forcefield
forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3pfb.xml'
)

# Create system
system = forcefield.createSystem(
    pdb.topology, 
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=HBonds
)

# Create constant temp integrator
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.004*picoseconds
)

# Create simulation and set initial positions
simulation = Simulation(
    pdb.topology,
    system,
    integrator
)
simulation.context.setPositions(pdb.positions)

# Local energy minimisation
simulation.minimizeEnergy()

# Create reporters for output
simulation.reporters.append(
    PDBReporter('output.pdb', 1000)
)
simulation.reporters.append(
    StateDataReporter(
        stdout, 
        1000, 
        step=True,
        potentialEnergy=True, 
        temperature=True
    )
)

In [ ]:
# Run the simulation through 10000 time steps
simulation.step(10000)

# Viewer

In [ ]:
import pytraj as pt
import nglview as nv
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
traj = pt.load("output.pdb")
view = nv.show_pytraj(traj)
view.add_representation("licorice")
view.add_unitcell()
view